# Coffee Shops in New York City

### This notebook will use Foursquare data alongside geo data in order to explore the coffee shops in Manhattan.  

### Part 1: Collecting the Data

In [1]:
#import libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#download data

!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
NYneighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [4]:
# complete the dataframe

for data in NYneighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
print('The NYC dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The NYC dataframe has 5 boroughs and 306 neighborhoods.


In [7]:
#Foursquare credentials

CLIENT_ID = 'BNEPNEJIRIKQUNA0NDIZB4AI4E54XQHB01BNXIGGNAFR05WD' # your Foursquare ID
CLIENT_SECRET = 'CM3YG2NVHR1K0GIHCVVTWS0PZ5ZTKJ0PB0WXJ1X2MJF3CQJ5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: BNEPNEJIRIKQUNA0NDIZB4AI4E54XQHB01BNXIGGNAFR05WD
CLIENT_SECRET:CM3YG2NVHR1K0GIHCVVTWS0PZ5ZTKJ0PB0WXJ1X2MJF3CQJ5


In [8]:
#Get latitude, longitude for foursquare

address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [9]:
# limit to Manhattan neighborhoods

manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [10]:
# get latitude, longitude for specific neighborhood

neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [11]:
#get venues within radius of latitude, longitude using Foursquare

LIMIT = 10000 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=BNEPNEJIRIKQUNA0NDIZB4AI4E54XQHB01BNXIGGNAFR05WD&client_secret=CM3YG2NVHR1K0GIHCVVTWS0PZ5ZTKJ0PB0WXJ1X2MJF3CQJ5&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=10000&limit=10000'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60b2989d1652f05c0a04705c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'New York',
  'headerFullLocation': 'New York',
  'headerLocationGranularity': 'city',
  'totalResults': 242,
  'suggestedBounds': {'ne': {'lat': 40.96655086879973,
    'lng': -73.79185326959468},
   'sw': {'lat': 40.78655068879955, 'lng': -74.02946604766494}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4baf59e8f964a520a6f93be3',
       'name': 'Bikram Yoga',
       'location': {'address': '5500 Broadway',
        'crossStreet': '230th Street',
        'lat': 40.876843690797934,
        'lng': -73.90620384419528,
        'label

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
# collect dataframe with venues, categories, latitude, longitude

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Bikram Yoga,Yoga Studio,40.876844,-73.906204
1,Sam's Pizza,Pizza Place,40.879435,-73.905859
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Kingsbridge Social Club,Pizza Place,40.884581,-73.901999
4,Inwood Hill Park,Park,40.870858,-73.920129


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [16]:
# function to get nearby venues from Foursquare

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
# use getNearbyVenues function to get venues in manhattan

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [18]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [19]:
# count venues in each neighborhood

manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,78,78,78,78,78,78
Carnegie Hill,96,96,96,96,96,96
Central Harlem,47,47,47,47,47,47
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,38,38,38,38,38,38
East Village,100,100,100,100,100,100


In [20]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 326 uniques categories.


### Now that we have a dataset with venues in Manhattan, it is possible to categorize the venues in order to understand frequency of each venue type within each neighborhood.

## Part 2: Digesting the Data and Graphing - Frequency of Coffee Shops as venues

In [21]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern 

In [22]:
# convert dataframe and group by neighborhood, and calculate frequency of each venue type through calculation of the mean

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern 

In [23]:
#return top 5 venue types for each neighborhood along with frequency

num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0            Park  0.10
1     Coffee Shop  0.06
2           Hotel  0.06
3  Clothing Store  0.05
4             Gym  0.04


----Carnegie Hill----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2    Wine Shop  0.04
3          Gym  0.03
4    Bookstore  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.06
1    Seafood Restaurant  0.06
2  Gym / Fitness Center  0.04
3   American Restaurant  0.04
4            Public Art  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.06
1          Art Gallery  0.05
2               Bakery  0.05
3  American Restaurant  0.04
4                Hotel  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.09
1               Bakery  0.07
2         Cocktail Bar  0.04
3  American Restaurant  0.04
4   Salon / Barbershop  0.03


----Civic Center----
                  venue  freq
0           Coffee Shop  0.07
1     

In [24]:
#function to return most common venue types

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
#create new dataframe to show most common venue types per neighborhood, to specified number, 10, in this case

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Clothing Store,Gym,Memorial Site,Playground,Pizza Place,Food Court,Plaza
1,Carnegie Hill,Coffee Shop,Café,Wine Shop,Gym,Bookstore,Pizza Place,French Restaurant,Bar,Yoga Studio,Italian Restaurant
2,Central Harlem,African Restaurant,Seafood Restaurant,Gym / Fitness Center,American Restaurant,Public Art,Bar,Art Gallery,French Restaurant,Chinese Restaurant,Caribbean Restaurant
3,Chelsea,Coffee Shop,Art Gallery,Bakery,American Restaurant,Hotel,French Restaurant,Wine Shop,Seafood Restaurant,Ice Cream Shop,Theater
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Salon / Barbershop,Spa,Hotpot Restaurant,Dessert Shop,Optical Shop,Boutique


### With this dataset, we can look qualitatively at the top most common venues to identify neighborhoods with high frequency of each venue. Here, we care about Coffee Shops.

### If an entreprenuer was looking to start a new Coffee Shop, they may care about which neighborhoods already have a lot of coffee shops.

In [26]:
#sort neighborhoods by Coffee Shop frequency

manhattan_coffee = manhattan_grouped.sort_values(by=['Coffee Shop'], ascending = False)
manhattan_coffee.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern 

In [27]:
#trim dataset to only show Coffee Shops

manhattan_coffee = manhattan_coffee[['Neighborhood','Coffee Shop']]
manhattan_coffee


,Neighborhood,Coffee Shop
9,Financial District,0.100000
30,Stuyvesant Town,0.095238
25,Morningside Heights,0.093023
22,Marble Hill,0.086957
21,Manhattanville,0.083333
28,Roosevelt Island,0.074074
1,Carnegie Hill,0.072917
5,Civic Center,0.070000
20,Manhattan Valley,0.066667
13,Hamilton Heights,0.066667


In [28]:
#merge dataset with latitude, longitude information for each neighborhood

manhattan_coffee_for_map = pd.merge(manhattan_coffee, manhattan_data,
                        how="left", on=["Neighborhood"])
manhattan_coffee_for_map.head()


,Neighborhood,Coffee Shop,Borough,Latitude,Longitude
0,Financial District,0.100000,Manhattan,40.707107,-74.010665
1,Stuyvesant Town,0.095238,Manhattan,40.731000,-73.974052
2,Morningside Heights,0.093023,Manhattan,40.808000,-73.963896
3,Marble Hill,0.086957,Manhattan,40.876551,-73.910660
4,Manhattanville,0.083333,Manhattan,40.816934,-73.957385


In [29]:
#remove Borough column

manhattan_coffee_for_map = manhattan_coffee_for_map[['Neighborhood','Coffee Shop','Latitude','Longitude']]
manhattan_coffee_for_map.head()

,Neighborhood,Coffee Shop,Latitude,Longitude
0,Financial District,0.100000,40.707107,-74.010665
1,Stuyvesant Town,0.095238,40.731000,-73.974052
2,Morningside Heights,0.093023,40.808000,-73.963896
3,Marble Hill,0.086957,40.876551,-73.910660
4,Manhattanville,0.083333,40.816934,-73.957385


In [30]:
manhattan_coffee_for_map

,Neighborhood,Coffee Shop,Latitude,Longitude
0,Financial District,0.100000,40.707107,-74.010665
1,Stuyvesant Town,0.095238,40.731000,-73.974052
2,Morningside Heights,0.093023,40.808000,-73.963896
3,Marble Hill,0.086957,40.876551,-73.910660
4,Manhattanville,0.083333,40.816934,-73.957385
5,Roosevelt Island,0.074074,40.762160,-73.949168
6,Carnegie Hill,0.072917,40.782683,-73.953256
7,Civic Center,0.070000,40.715229,-74.005415
8,Manhattan Valley,0.066667,40.797307,-73.964286
9,Hamilton Heights,0.066667,40.823604,-73.949688


In [31]:
#install proper packages for map building

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium
import pandas as pd
import folium.plugins
import branca
import branca.colormap as cm

# create map of Manhattan using latitude and longitude values
map_manhattan_coffee = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=11)

#use colormap to colorcode neighborhoods based on frequency of Coffee Shops
colormap = cm.LinearColormap(colors=['green','yellow','red'], index=[0,0.06,0.12],vmin=0,vmax=0.12)

# add markers to map
for lat, lng, label, p in zip(manhattan_coffee_for_map['Latitude'], manhattan_coffee_for_map['Longitude'], manhattan_coffee_for_map['Neighborhood'], manhattan_coffee_for_map['Coffee Shop']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colormap(p),
        fill=True,
        #fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_coffee)  

map_manhattan_coffee 

Solving environment: done

# All requested packages already installed.



In [32]:
manhattan_coffee_for_map.head()



,Neighborhood,Coffee Shop,Latitude,Longitude
0,Financial District,0.100000,40.707107,-74.010665
1,Stuyvesant Town,0.095238,40.731000,-73.974052
2,Morningside Heights,0.093023,40.808000,-73.963896
3,Marble Hill,0.086957,40.876551,-73.910660
4,Manhattanville,0.083333,40.816934,-73.957385


In [33]:

most_coffee_shops = manhattan_coffee_for_map.iloc[0:3]["Neighborhood"].tolist()
print(most_coffee_shops)



['Financial District', 'Stuyvesant Town', 'Morningside Heights']


In [34]:
least_coffee_shops = manhattan_coffee_for_map.iloc[-3:]["Neighborhood"].tolist()
print(least_coffee_shops)

['Chinatown', 'Inwood', 'East Harlem']


In [35]:
print("As you can see, the neighborhoods with the lowest frequency of coffee shops are: ",least_coffee_shops)
print("As you can see, the neighborhoods with the highest frequency of coffee shops are: ",most_coffee_shops)

As you can see, the neighborhoods with the lowest frequency of coffee shops are:  ['Chinatown', 'Inwood', 'East Harlem']
As you can see, the neighborhoods with the highest frequency of coffee shops are:  ['Financial District', 'Stuyvesant Town', 'Morningside Heights']


### While this information may be helpful as a first step, not all neighborhoods are the same size. Perhaps it is also important to look at which neighborhoods have the highest number of coffee shops per person.

## Part 3: Digesting the Data and Graphing - Normalizing data based on population

In [36]:
LIMIT = 10000 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
search_query = 'Coffee Shop'
# create URL
url2 = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    search_query,
    radius, 
    LIMIT)
url2 # display URL

'https://api.foursquare.com/v2/venues/search?&client_id=BNEPNEJIRIKQUNA0NDIZB4AI4E54XQHB01BNXIGGNAFR05WD&client_secret=CM3YG2NVHR1K0GIHCVVTWS0PZ5ZTKJ0PB0WXJ1X2MJF3CQJ5&v=20180605&ll=40.87655077879964,-73.91065965862981&query=Coffee Shop&radius=10000&limit=10000'

In [37]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '60b298cf4162014f099ef5c8'},
 'response': {'venues': [{'id': '4be88652947820a1e802b5db',
    'name': "G's Coffee Shop",
    'location': {'address': '634 W 207th St',
     'crossStreet': 'btwn Broadway & Cooper St',
     'lat': 40.86841792395646,
     'lng': -73.92174351734803,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.86841792395646,
       'lng': -73.92174351734803}],
     'distance': 1300,
     'postalCode': '10034',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['634 W 207th St (btwn Broadway & Cooper St)',
      'New York, NY 10034',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d147941735',
      'name': 'Diner',
      'pluralName': 'Diners',
      'shortName': 'Diner',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/diner_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1622317264',
  

In [38]:
# Instead of frequency, this time group venues by Neighborhood, and add a count

manhattan_coffee_grouped = manhattan_onehot.groupby('Neighborhood').sum().reset_index()
manhattan_coffee_grouped

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern 

In [39]:
# sort by number of coffee shops

manhattan_coffee_grouped = manhattan_coffee_grouped.sort_values(by=['Coffee Shop'], ascending = False)
manhattan_coffee_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern 

In [40]:
#trim dataset for only neighborhood and coffee shop columns

manhattan_coffee_grouped2 = manhattan_coffee_grouped[['Neighborhood','Coffee Shop']]
manhattan_coffee_grouped2

,Neighborhood,Coffee Shop
9,Financial District,10
5,Civic Center,7
1,Carnegie Hill,7
3,Chelsea,6
16,Lenox Hill,6
0,Battery Park City,5
35,Upper East Side,5
34,Turtle Bay,5
26,Murray Hill,5
39,Yorkville,5


In [41]:
#download nyc neighborhood population data

import io
#!wget -q -O 'newyork_popdata.csv' https://github.com/afigatner/Coursera_Capstone/blob/master/NYCPopulationData.csv
#newyork_popdata = pd.read_csv('newyork_popdata.csv',error_bad_lines=False)
#!wget -q -O 'newyork_popdata.xls' https://github.com/afigatner/Coursera_Capstone/blob/master/NYCPopulationData.xls
#newyork_popdataexcel = pd.read_excel('newyork_popdata.xls')
#newyork_popdataexcel

url3 = "https://raw.githubusercontent.com/afigatner/Coursera_Capstone/master/NYCPopulationData.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url3).content

# Reading the downloaded content and turning it into a pandas dataframe

newyork_popdata = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [42]:
newyork_popdata

,Rank,Neighborhood,Population
0,1,Midtown,"391,371"
1,2,Lower Manhattan,"382,654"
2,3,Harlem,"335,109"
3,4,Upper East Side,"229,688"
4,5,Upper West Side,"209,084"
5,6,Washington Heights,"158,318"
6,7,East Harlem,"115,921"
7,8,Chinatown,"100,000"
8,9,Lower East Village,"72,957"
9,10,Alphabet City,"63,347"


In [43]:
#combine datasets

merge_coffeeshops_population = pd.merge(manhattan_coffee_grouped2, newyork_popdata, how = "outer", on = ["Neighborhood"])
merge_coffeeshops_population

,Neighborhood,Coffee Shop,Rank,Population
0,Financial District,10.0,13.0,"60,976"
1,Civic Center,7.0,NaN,NaN
2,Carnegie Hill,7.0,NaN,NaN
3,Chelsea,6.0,19.0,"38,242"
4,Lenox Hill,6.0,NaN,NaN
5,Battery Park City,5.0,18.0,"39,699"
6,Upper East Side,5.0,4.0,"229,688"
7,Turtle Bay,5.0,NaN,NaN
8,Murray Hill,5.0,30.0,"10,284"
9,Yorkville,5.0,20.0,"35,221"


In [44]:
#sort by Neighborhood in order to identify areas of overlap between datasets

merge_coffeeshops_population2 = merge_coffeeshops_population.sort_values(by=['Neighborhood'], ascending = False)
merge_coffeeshops_population2

,Neighborhood,Coffee Shop,Rank,Population
9,Yorkville,5.0,20.0,"35,221"
18,West Village,3.0,NaN,NaN
36,Washington Heights,2.0,6.0,"158,318"
19,Upper West Side,3.0,5.0,"209,084"
6,Upper East Side,5.0,4.0,"229,688"
46,Two Bridges,NaN,22.0,"28,915"
7,Turtle Bay,5.0,NaN,NaN
20,Tudor City,3.0,NaN,NaN
34,Tribeca,2.0,29.0,"17,362"
13,Sutton Place,4.0,NaN,NaN


In [45]:
#clean data to better merge datasets

newyork_popdata_edit = newyork_popdata
newyork_popdata_edit['Neighborhood']= newyork_popdata_edit['Neighborhood'].replace(['Koreatown','Meatpacking Ditrict','Two Bridges','SoHo','NoHo','Harlem','Lower Manhattan','Hell\'s Kitchen','Flatiron District','Gramercy Park','Hudson Heights'],['Midtown South','West Village','Lower East Side','Soho','Noho','Central Harlem','Civic Center','Clinton','Flatiron','Gramercy','Hudson Yards'])
newyork_popdata_edit

,Rank,Neighborhood,Population
0,1,Midtown,"391,371"
1,2,Civic Center,"382,654"
2,3,Central Harlem,"335,109"
3,4,Upper East Side,"229,688"
4,5,Upper West Side,"209,084"
5,6,Washington Heights,"158,318"
6,7,East Harlem,"115,921"
7,8,Chinatown,"100,000"
8,9,Lower East Village,"72,957"
9,10,Alphabet City,"63,347"


In [46]:
#clean data to better merge datasets 

manhattan_coffee_edit = manhattan_coffee_grouped2
manhattan_coffee_edit['Neighborhood']= manhattan_coffee_edit['Neighborhood'].replace(['Manhattanville','Roosevelt Island','Sutton Place','Tudor City','Turtle Bay','Manhattan Valley','Lenox Hill','Carnegie Hill'],['Harlem','Lower East Side','Kips Bay','Kips Bay','Kips Bay','Upper West Side', 'Upper East Side','Upper East Side'])
manhattan_coffee_edit

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Neighborhood,Coffee Shop
9,Financial District,10
5,Civic Center,7
1,Upper East Side,7
3,Chelsea,6
16,Upper East Side,6
0,Battery Park City,5
35,Upper East Side,5
34,Kips Bay,5
26,Murray Hill,5
39,Yorkville,5


In [47]:
#find duplicates and clean data by combining neighborhoods
manhattan_coffee_edit_group = manhattan_coffee_edit.groupby('Neighborhood').sum().reset_index()
manhattan_coffee_edit_group

,Neighborhood,Coffee Shop
0,Battery Park City,5
1,Central Harlem,1
2,Chelsea,6
3,Chinatown,2
4,Civic Center,7
5,Clinton,4
6,East Harlem,0
7,East Village,3
8,Financial District,10
9,Flatiron,3


In [48]:
#find duplicates and clean data by combining neighborhoods

newyork_popdata_edit2 = newyork_popdata_edit[['Neighborhood','Population']]
newyork_popdata_edit_group2 = newyork_popdata_edit2.groupby('Neighborhood').sum().reset_index()
newyork_popdata_edit_group2

,Neighborhood,Population
0,Alphabet City,"63,347"
1,Battery Park City,"39,699"
2,Central Harlem,"335,109"
3,Chelsea,"38,242"
4,Chinatown,"100,000"
5,Civic Center,"382,654"
6,Clinton,"45,884"
7,East Harlem,"115,921"
8,East Village,"62,832"
9,Financial District,"60,976"


In [49]:
# merge cleaned datasets

edited_merge_coffeeshops_population = pd.merge(manhattan_coffee_edit_group, newyork_popdata_edit_group2, how = "outer", on = ["Neighborhood"])
sorted_merge_coffeeshops_population = edited_merge_coffeeshops_population.dropna()
sorted_merge_coffeeshops_population = sorted_merge_coffeeshops_population.sort_values(by=['Coffee Shop'], ascending = False)
sorted_merge_coffeeshops_population

,Neighborhood,Coffee Shop,Population
29,Upper East Side,18.0,"229,688"
16,Kips Bay,12.0,"5,330"
8,Financial District,10.0,"60,976"
4,Civic Center,7.0,"382,654"
2,Chelsea,6.0,"38,242"
30,Upper West Side,6.0,"209,084"
0,Battery Park City,5.0,"39,699"
24,Murray Hill,5.0,"10,284"
33,Yorkville,5.0,"35,221"
21,Midtown,4.0,"391,371"


In [50]:
# Calculate number of people per coffee shop in each neighborhood, sort by this value

sorted_merge_coffeeshops_population2 = sorted_merge_coffeeshops_population
sorted_merge_coffeeshops_population2['Population'] = sorted_merge_coffeeshops_population2['Population'].str.replace(",","").astype(float)
sorted_merge_coffeeshops_population2['Population per shop'] = sorted_merge_coffeeshops_population2['Population'] / sorted_merge_coffeeshops_population2['Coffee Shop']
sorted_merge_coffeeshops_population3 = sorted_merge_coffeeshops_population2
sorted_merge_coffeeshops_population3['Population per shop'] = sorted_merge_coffeeshops_population3['Population per shop'].astype(float)
sorted_merge_coffeeshops_population3['Population per shop'] = sorted_merge_coffeeshops_population3['Population per shop'].replace([np.inf, -np.inf],0)
sorted_merge_coffeeshops_population3 = sorted_merge_coffeeshops_population3.sort_values(by=['Population per shop'], ascending = False)
sorted_merge_coffeeshops_population3

,Neighborhood,Coffee Shop,Population,Population per shop
1,Central Harlem,1.0,335109.0,335109.000000
21,Midtown,4.0,391371.0,97842.750000
31,Washington Heights,2.0,158318.0,79159.000000
4,Civic Center,7.0,382654.0,54664.857143
3,Chinatown,2.0,100000.0,50000.000000
15,Inwood,1.0,46746.0,46746.000000
30,Upper West Side,6.0,209084.0,34847.333333
7,East Village,3.0,62832.0,20944.000000
17,Lincoln Square,3.0,61489.0,20496.333333
23,Morningside Heights,4.0,55929.0,13982.250000


In [51]:
#add geo data for map

data_for_population_map = pd.merge(sorted_merge_coffeeshops_population3, manhattan_coffee_for_map, how = "left", on = ["Neighborhood"])
data_for_population_map = data_for_population_map.dropna()
data_for_population_map

,Neighborhood,Coffee Shop_x,Population,Population per shop,Coffee Shop_y,Latitude,Longitude
0,Central Harlem,1.0,335109.0,335109.000000,0.021277,40.815976,-73.943211
1,Midtown,4.0,391371.0,97842.750000,0.040000,40.754691,-73.981669
2,Washington Heights,2.0,158318.0,79159.000000,0.023529,40.851903,-73.936900
3,Civic Center,7.0,382654.0,54664.857143,0.070000,40.715229,-74.005415
4,Chinatown,2.0,100000.0,50000.000000,0.020000,40.715618,-73.994279
5,Inwood,1.0,46746.0,46746.000000,0.018519,40.867684,-73.921210
6,Upper West Side,6.0,209084.0,34847.333333,0.030000,40.787658,-73.977059
7,East Village,3.0,62832.0,20944.000000,0.030000,40.727847,-73.982226
8,Lincoln Square,3.0,61489.0,20496.333333,0.030303,40.773529,-73.985338
9,Morningside Heights,4.0,55929.0,13982.250000,0.093023,40.808000,-73.963896


In [52]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium
#import pandas as pd
#import folium.plugins
#import branca
#import branca.colormap as cm

# create map of Manhattan using latitude and longitude values
map_manhattan_coffee_population = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=11)

colormap2 = cm.LinearColormap(colors=['red','yellow','green'], index=[0,7000,15000],vmin=0,vmax=100000)

# add markers to map
for lat, lng, label, p in zip(data_for_population_map['Latitude'], data_for_population_map['Longitude'], data_for_population_map['Neighborhood'], data_for_population_map['Population per shop']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colormap2(p),
        fill=True,
        #fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_coffee_population)  

map_manhattan_coffee_population

In [53]:
most_coffee_shops_per_person = data_for_population_map.iloc[0:3]["Neighborhood"].tolist()
print(most_coffee_shops_per_person)
least_coffee_shops_per_person = data_for_population_map.iloc[-3:]["Neighborhood"].tolist()
print(least_coffee_shops_per_person)

['Central Harlem', 'Midtown', 'Washington Heights']
['Murray Hill', 'Little Italy', 'East Harlem']


In [54]:
print("As you can see, the neighborhoods with the most people per coffee shop are: ", most_coffee_shops_per_person)
print("As you can see, the neighborhoods with the least people per coffee shop are: ", least_coffee_shops_per_person)

As you can see, the neighborhoods with the most people per coffee shop are:  ['Central Harlem', 'Midtown', 'Washington Heights']
As you can see, the neighborhoods with the least people per coffee shop are:  ['Murray Hill', 'Little Italy', 'East Harlem']


### As you can see, from Part 2 we now have a list of neighborhoods where coffee shops are the most/least frequent venues. And from Part 3 we have a list of neighborhoods where there is a high/low ratio of people per coffee shop. 

### In addition to the current state of a neighborhood, it may be useful to also take into account neighborhoods with highest growth from year to year. Neighborhoods with highest growth could generate larger revenues in the future.

## Part 4: Integrating Neighborhood Growth

In [55]:
url5 = "https://raw.githubusercontent.com/afigatner/Coursera_Capstone/master/ManhattanGrowthData2.csv" # Make sure the url is the raw version of the file on GitHub
growth_download = requests.get(url5).content
nyc_neighborhood_growth_data = pd.read_csv(io.StringIO(growth_download.decode('utf-8')))
nyc_neighborhood_growth_data


,Rank,Borough,Neighborhood,Median Sale Price 2020,Y-o-Y Change in Median Sale Price,No of Transactions 2020,Y-o-Y Change in No of Transactions
0,1,Manhattan,Hudson Yards,"$4,504,000.0",15%,23,-85%
1,2,Manhattan,TriBeCa,"$3,157,000.0",-3%,198,-51%
2,3,Manhattan,Little Italy,"$2,750,000.0",4%,35,25%
3,4,Manhattan,SoHo,"$2,463,000.0",-10%,64,-45%
4,5,Manhattan,Hudson Square,"$2,100,000.0",-15%,78,-26%
5,6,Manhattan,Theatre District - Times Square,"$1,807,000.0",8%,127,-39%
6,7,Manhattan,Central Midtown,"$1,790,000.0",44%,225,3%
7,8,Brooklyn,DUMBO,"$1,625,000.0",8%,38,-31%
8,9,Manhattan,Flatiron District,"$1,593,000.0",-17%,178,-51%
9,10,Brooklyn,Cobble Hill,"$1,491,000.0",-9%,44,-53%


In [56]:
# clean data 

nyc_neighborhood_growth_data = nyc_neighborhood_growth_data.drop(nyc_neighborhood_growth_data.index[29])
nyc_neighborhood_growth_data = nyc_neighborhood_growth_data.drop(nyc_neighborhood_growth_data.index[17])
nyc_neighborhood_growth_data

,Rank,Borough,Neighborhood,Median Sale Price 2020,Y-o-Y Change in Median Sale Price,No of Transactions 2020,Y-o-Y Change in No of Transactions
0,1,Manhattan,Hudson Yards,"$4,504,000.0",15%,23,-85%
1,2,Manhattan,TriBeCa,"$3,157,000.0",-3%,198,-51%
2,3,Manhattan,Little Italy,"$2,750,000.0",4%,35,25%
3,4,Manhattan,SoHo,"$2,463,000.0",-10%,64,-45%
4,5,Manhattan,Hudson Square,"$2,100,000.0",-15%,78,-26%
5,6,Manhattan,Theatre District - Times Square,"$1,807,000.0",8%,127,-39%
6,7,Manhattan,Central Midtown,"$1,790,000.0",44%,225,3%
7,8,Brooklyn,DUMBO,"$1,625,000.0",8%,38,-31%
8,9,Manhattan,Flatiron District,"$1,593,000.0",-17%,178,-51%
9,10,Brooklyn,Cobble Hill,"$1,491,000.0",-9%,44,-53%


In [57]:
# sort by year on year change in number of transactions

nyc_neighborhood_growth_data['Y-o-Y Change in No of Transactions'] = nyc_neighborhood_growth_data['Y-o-Y Change in No of Transactions'].str.replace("%", "").astype(int)
nyc_neighborhood_growth_data = nyc_neighborhood_growth_data.sort_values(by=['Y-o-Y Change in No of Transactions'], ascending = False)


In [58]:
nyc_neighborhood_growth_data_manhattan = nyc_neighborhood_growth_data[nyc_neighborhood_growth_data['Borough'] == 'Manhattan'] 
nyc_neighborhood_growth_data_manhattan

,Rank,Borough,Neighborhood,Median Sale Price 2020,Y-o-Y Change in Median Sale Price,No of Transactions 2020,Y-o-Y Change in No of Transactions
2,3,Manhattan,Little Italy,"$2,750,000.0",4%,35,25
6,7,Manhattan,Central Midtown,"$1,790,000.0",44%,225,3
23,24,Manhattan,Financial District,"$1,100,000.0",10%,212,-18
36,34,Manhattan,Sutton Place,"$900,000.0",-16%,160,-20
20,21,Manhattan,Greenwich Village,"$1,136,000.0",-19%,368,-24
12,13,Manhattan,Upper West Side,"$1,303,000.0",18%,1429,-25
4,5,Manhattan,Hudson Square,"$2,100,000.0",-15%,78,-26
14,15,Manhattan,West Village,"$1,269,000.0",1%,183,-27
41,39,Manhattan,Turtle Bay,"$873,000.0",3%,214,-30
28,27,Manhattan,Upper East Side,"$1,050,000.0",-11%,1457,-31


In [59]:
high_percent_change_in_transactions = nyc_neighborhood_growth_data_manhattan.iloc[0:3]["Neighborhood"].tolist()
print(high_percent_change_in_transactions)
low_percent_change_in_transactions = nyc_neighborhood_growth_data_manhattan.iloc[-3:]["Neighborhood"].tolist()
print(low_percent_change_in_transactions)

['Little Italy', 'Central Midtown', 'Financial District']
['Chinatown', 'Lower East Side', 'Hudson Yards']


In [60]:
print("As you can see, the neighborhoods with the highest growth in transactions in 2020: ", high_percent_change_in_transactions)
print("As you can see, the neighborhoods with the lowest growth in transactions in 2020: ", low_percent_change_in_transactions)

As you can see, the neighborhoods with the highest growth in transactions in 2020:  ['Little Italy', 'Central Midtown', 'Financial District']
As you can see, the neighborhoods with the lowest growth in transactions in 2020:  ['Chinatown', 'Lower East Side', 'Hudson Yards']


### When considering neighborhoods to open a new coffee shop, the ideal scenario would be to find a neighborhood with fewer coffee shops and high potential (high growth). 

In [61]:
print("The neighborhoods with the highest growth in transactions in 2020: ", high_percent_change_in_transactions)
print("The neighborhoods with the lowest frequency of coffee shops are: ",least_coffee_shops)
print("The neighborhoods with the most people per coffee shop are: ", most_coffee_shops_per_person)


The neighborhoods with the highest growth in transactions in 2020:  ['Little Italy', 'Central Midtown', 'Financial District']
The neighborhoods with the lowest frequency of coffee shops are:  ['Chinatown', 'Inwood', 'East Harlem']
The neighborhoods with the most people per coffee shop are:  ['Central Harlem', 'Midtown', 'Washington Heights']


### Any of these analyses could be used to assist in decision-making. While there are no clear winners, since Midtown shows up in two out of three assessments, perhaps this would be an appropriate choice.